In [1]:
import os
import re
import json
import time
import pickle
import pandas as pd
from openai import OpenAI
import os
import pandas as pd
import json
import time

API_KEY = 'sk-bedceae2ceba437f944db22706354095'

In [2]:
flight_options='''
Option 1:

 Indigo
 
6E5065
Departs from Calcutta, 02:00, 10 August 2025.
02:00
CCU  Calcutta
Flight duration 2 hours 50 minutes
2 hours 50
Arrives in Mumbai, 04:50, August 10, 2025.
04:50
BOOM  Bombay
Connection at the airport. 2 hours 25 minutes between flights.
2 hours 25
Connect at the airport
Flight segment 1 with IndiGo. Flight number 6E17. Operated by SmartLynx.

 Indigo
 
6E17
| Operated by SmartLynx
Departs Mumbai, 07:15, 10 August 2025.
07:15
BOOM  Bombay
Flight duration 6 hours 50 minutes
6 hours 50
Arrives in Istanbul, 11:35, August 10, 2025.
11:35
Istanbul ​
Long layover. 3 hours 20 minutes between flights.
3 hours 20
Connect at the airportLong wait
Flight segment 1 with IndiGo. Flight number 6E4265. Operated by Turkish Airlines.

 Indigo
 
6E4265
| Operated by Turkish Airlines
Departs Istanbul, 14:55, August 10, 2025.
14:55
Istanbul ​
Flight duration 3 hours 40 minutes
3 hours 40
Arrives at Stockholm Arlanda, 17:35, August 10, 2025.
17:35
ARN  Stockholm Arlanda
5,318 SEK

Option 2:

 Lufthansa
 
LH5315
| Operated by Air India
Departs from Calcutta, 20:30, 10 August 2025.
20:30
CCU  Calcutta
Flight duration 2 hours 35 minutes
2 hours 35
Arrives at Indira Gandhi International, 23:05, 10 August 2025.
23:05
PART  Indira Gandhi International
Connection at the airport. 2 hours 40 minutes between flights.
2 hours 40
Connect at the airport
Flight segment 1 with Lufthansa. Flight number LH763.

 Lufthansa
 
LH763
Departs from Indira Gandhi International, 01:45, 11 August 2025.
01:45
+1
PART  Indira Gandhi International
Flight duration 8 hours 15 minutes
8 hours 15
Arrives in Munich, 06:30, August 11, 2025.
06:30
MUC  Munich
Connection at the airport. 2 hours 20 minutes between flights.
2 hours 20
Connect at the airport
Flight segment 1 with Lufthansa. Flight number LH2414.

 Lufthansa
 
LH2414
Departs Munich, 08:50, August 11, 2025.
08:50
MUC  Munich
Flight duration 2 hours 10 minutes
2 hours 10
Arrives at Stockholm Arlanda, 11:00, August 11, 2025.
11:00
ARN  Stockholm Arlanda
Flight leg 1 arrives on Monday, August 11, 2025.Arrives: Mon 11 Aug 2025Travel time: 18 hoursTravel time: 18 hours

6327 SEK


Option 3:


 British Airways
 
BA7954
| Operated by IndiGo
Departs from Kolkata, 06:20, 10 August 2025.
06:20
CCU  Calcutta
Flight duration 2 hours 50 minutes
2 hours 50
Arrives in Mumbai, 09:10, August 10, 2025.
09:10
BOOM  Bombay
Long layover. 3 hours 55 minutes between flights.
3 hours 55
Connect at the airportLong wait
Flight segment 1 with British Airways. Flight number BA198.

 British Airways
 
BA198
Departs Mumbai, 13:05, 10 August 2025.
13:05
BOOM  Bombay
Flight duration 9 hours 45 minutes
9 hours 45
Arrives at London Heathrow, 18:20, 10 August 2025.
18:20
LHR  London Heathrow
Connection at the airport. 2 hours 15 minutes between flights.
2 hours 15
Connect at the airport
Flight segment 1 with British Airways. Flight number BA786.

 British Airways
 
BA786
Departs London Heathrow, 20:35, 10 August 2025.
20:35
LHR  London Heathrow
Flight duration 2 hours 30 minutes
2 hours 30
Arrives at Stockholm Arlanda, 00:05, August 11, 2025. Arrives one day later.
00:05
+1
ARN  Stockholm Arlanda
Flight leg 1 arrives on Monday, August 11, 2025.Arrives: Mon 11 Aug 2025Journey time: 21 hours 15 minutesJourney time: 21 hours 15 minutes

6022 SEK


Option 4:

 Emirates
 
EK571
Departs from Kolkata, 09:45, 10 August 2025.
09:45
CCU  Calcutta
Flight duration 4 hours 50 minutes
4 hours 50
Arrives in Dubai, 13:05, August 10, 2025.
13:05
Dubai ​
Connection at the airport. 1 hour 10 minutes between flights.
1 hour 10
Connect at the airport
Flight segment 1 with Emirates. Flight number EK67.

 Emirates
 
EK67
Departs Dubai, 14:15, August 10, 2025.
14:15
Dubai ​
Flight duration 7 hours 30 minutes
7 hours 30
Arrives at London Stansted, 18:45, August 10, 2025.
18:45
STN  London Stansted
Connection at the airport. 1 hour 15 minutes between flights.
1 hour 15
Connect at the airport
Flight segment 1 with Ryanair. Flight number FR1463. Operated by Malta Air.
 Ryanair
 
FR1463
| Operated by Malta Air
Departs London Stansted, 8:00 PM, August 10, 2025.
20:00
STN  London Stansted
Flight duration 2 hours 20 minutes
2 hours 20
Arrives at Stockholm Arlanda, 23:20, August 10, 2025.
23:20
ARN  Stockholm Arlanda
Flight leg 1 arrives on Sunday, August 10, 2025.Arrives: Sun 10 Aug 2025Travel time: 17 hours 5 minutesTravel time: 17 hours 05
6422 SEK

Option 5:


 Turkish Airlines
 
TK4703
| Operated by IndiGo
Departs from Kolkata, 10:30 PM, August 10, 2025.
22:30
CCU  Calcutta
Flight duration 2 hours 30 minutes
2 hours 30
Arrives at Indira Gandhi International, 01:00, 11 August 2025. Arrives one day later.
01:00
+1
PART  Indira Gandhi International
Long layover. 5 hours 40 minutes between flights.
5 hours 40
Connect at the airportLong wait
Flight segment 1 with Turkish Airlines. Flight number TK4650. Operated by SmartLynx.

 Turkish Airlines
 
TK4650
| Operated by SmartLynx
Departs from Indira Gandhi International, 06:40, 11 August 2025.
06:40
PART  Indira Gandhi International
Flight duration 7 hours 40 minutes
7 hours 40
Arrives in Istanbul, 11:50, August 11, 2025.
11:50
Istanbul ​
Long layover. 3 hours 5 minutes between flights.
3 hours 05
Connect at the airportLong wait
Flight segment 1 with Turkish Airlines. Flight number TK1795.

 Turkish Airlines
 
TK1795
Departs Istanbul, 14:55, August 11, 2025.
14:55
Istanbul ​
Flight duration 3 hours 40 minutes
3 hours 40
Arrives at Stockholm Arlanda, 17:35, August 11, 2025.
17:35
ARN  Stockholm Arlanda
Flight leg 1 arrives on Monday, August 11, 2025.Arrives: Mon 11 Aug 2025Journey time: 22 hours 35 minutesJourney time: 22 hours 35 minutes

6152 SEK
'''


In [7]:
# user_prompt='traveling with aged parents, dont mind long waits but prefer not having to change terminals'
user_prompt='''traveling alone, cheap flights preferred 
dislike: too long layovers 
preference: international flight at the start'''
system_prompt = f"""
You are an expert travel agent

You will get a list of travel itenerary and you need to select the best flight as per the users preference.

Make sure you select the best flight only from the list provided.
Flight options list: {flight_options}
"""


In [8]:
api_client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

In [9]:
response = api_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    #response_format={'type': 'json_object'}
)

In [10]:
print(response.choices[0].message.content)

Based on your preferences (traveling alone, preferring cheap flights, disliking long layovers, and preferring an international flight at the start), here's the best option from the list:

### **Best Option: Option 4 (Emirates)**
- **Price:** 6,422 SEK (cheaper than Lufthansa and Turkish Airlines options)
- **Total Travel Time:** 17 hours 5 minutes (reasonable compared to others)
- **Layovers:** 
  - Dubai: 1h 10m (short and efficient)
  - London Stansted: 1h 15m (short and efficient)
- **International Flight First:** Yes (CCU → DXB is the first leg)
- **No Excessively Long Layovers:** Both layovers are under 2 hours.

### **Why Not Other Options?**
1. **Option 1 (Indigo)** – Long layovers (3h 20m in Istanbul), not the cheapest.
2. **Option 2 (Lufthansa)** – More expensive (6,327 SEK), longer total travel time (18h).
3. **Option 3 (British Airways)** – Very long travel time (21h 15m), expensive (6,022 SEK).
4. **Option 5 (Turkish Airlines)** – Long layovers (5h 40m in Delhi, 3h 5m in Is